In [ ]:
import pandas as pd
import difflib
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv('../data/creative_services.csv', skiprows=4)

In [ ]:
df.head()

In [ ]:
df.drop(0, inplace=True)

In [ ]:
df.rename(columns={'Unnamed: 0': 'Country Name', 'YEAR': 'Category'}, inplace=True)

In [ ]:
import difflib
countries = pd.read_csv('../data/country-codes_csv.csv')
def get_country_iso(name):
    p = difflib.get_close_matches(name, countries['official_name_en'].dropna().unique(), n=1, cutoff=0.5)
    if len(p)>0:
        return countries[countries['official_name_en']==p[0]]['ISO3166-1-Alpha-3'].to_list()[0]
    else:
        return ""

In [ ]:
df['country_iso'] = [get_country_iso(x) for x in df['Country Name']]
df[df['country_iso']=='']

For most datasets, we are considering only Panama. So we will drop its subdivisions:

In [ ]:
df[df['Country Name'].str.contains('Panama')].groupby('Country Name')['country_iso'].last()

In [ ]:
df.count()

In [ ]:
df.drop(df[df['Country Name'].str.contains('Panama,')].index, inplace=True)

In [ ]:
df.count()

In [ ]:
df.columns[2:-1]

In [ ]:
df_new = df[['country_iso', 'Category', *df.columns[2:-1]]]

In [ ]:
df_new_2 = df_new.melt(id_vars=["country_iso", "Category"], var_name="year", value_name="value")

In [ ]:
df_new_2['value'].value_counts()

In [ ]:
import numpy as np
df_new_2['value'].replace('..', np.nan, inplace=True)
df_new_2['value'].replace('-', np.nan, inplace=True)
df_new_2['value'].replace('_', np.nan, inplace=True)
df_new_2['value'].value_counts()

In [ ]:
df_new_3 = df_new_2[['country_iso', 'year', 'Category', 'value']]

In [ ]:
df_new_3['tmp_idx'] = df_new_3.index

In [ ]:
df_new_3.dropna(inplace=True)

In [ ]:
df_new_3

In [ ]:
df_new_4 = df_new_3.pivot(index='tmp_idx', columns='Category', values='value')

In [ ]:
df_new_4

In [ ]:
df_new_4['country_iso'] = df_new_3['country_iso']
df_new_4['year'] = df_new_3['year']
df_new_4

In [ ]:
df_new_4.columns

In [ ]:
df_new_4.rename(columns={
    '  Audiovisual and related services': 'creative_svc_audiovisual',
    '  Other other personal, cultural and recreational services': 'creative_svc_other_personal_cultural_recreational',
    'Advertising, market research and public opinion polling': 'creative_svc_advertising_mktresearch_polling',
    'Architectural, engineering and other technical services': 'creative_svc_architectural_engineering_technical',
    'Personal, cultural and recreational services': 'creative_svc_personal_cultural_recreational',
    'Research and Development': 'creative_svc_research'
}, inplace=True)

In [ ]:
df_new_4.columns

In [ ]:
df_new_5 = df_new_4[['country_iso', 'year', *df_new_4.columns[0:6]]]
df_new_5

In [ ]:
df_new_5.to_csv('../data/converted/creative_services.csv', index=False)

In [ ]:
df_saved = pd.read_csv('../data/converted/creative_services.csv')
df_saved